In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

    

import torchvision
import torchvision.transforms as transforms

from models import *   # bring everything in the folder models

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128

#model_name = "cifar10"
#model = cifar10()
model_name_resnet20 = "resnet20"
model_resnet20 = resnet20_cifar()

model_name_vgg16 = "vgg16"
model_vgg16 = VGG16()       

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()   ## at the begining of each epoch, this should be reset
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()  # measure current time
    
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)  # data loading time

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end) # time spent to process one batch
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n    ## n is impact factor
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [2]:

fdir = 'result/'+str(model_name_resnet20)+'/model_best.pth.tar'

checkpoint = torch.load(fdir)
model_resnet20.load_state_dict(checkpoint['state_dict'])


criterion = nn.CrossEntropyLoss().cuda()

model_resnet20.eval()
model_resnet20.cuda()


prec = validate(testloader, model_resnet20, criterion)

Test: [0/79]	Time 0.219 (0.219)	Loss 0.3048 (0.3048)	Prec 90.625% (90.625%)
 * Prec 91.150% 


In [3]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output_resnet20 = SaveOutput()

In [14]:
for layer in model_resnet20.modules():
    print(layer)

ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2

In [4]:
for layer in model_resnet20.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.register_forward_pre_hook(save_output_resnet20)       ## Input for the module will be grapped       
####################################################
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu") 

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model_resnet20(images)

print("1st convolution's input size:", save_output_resnet20.outputs[0][0].size())
print("2nd convolution's input size:", save_output_resnet20.outputs[1][0].size())

1st convolution's input size: torch.Size([128, 3, 32, 32])
2nd convolution's input size: torch.Size([128, 16, 32, 32])


In [5]:
for layer in model_resnet20.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print(layer)

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
QuantConv2d(
  32, 32, kernel_size=(3, 3)

In [6]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
   

import torchvision
import torchvision.transforms as transforms

from models import *



def act_quantization(b):

    def uniform_quant(x, b=3):
        xdiv = x.mul(2 ** b - 1)
        xhard = xdiv.round().div(2 ** b - 1)
        return xhard

    class uq(torch.autograd.Function):   # here single underscore means this class is for internal use

        def forward(ctx, input, alpha):
            input_d = input/alpha
            input_c = input_d.clamp(max=1)  # Mingu edited for Alexnet
            input_q = uniform_quant(input_c, b)
            ctx.save_for_backward(input, input_q)
            input_q_out = input_q.mul(alpha)
            return input_q_out

    return uq().apply




def weight_quantization(b):

    def uniform_quant(x, b):
        xdiv = x.mul((2 ** b - 1))
        xhard = xdiv.round().div(2 ** b - 1)  
        return xhard

    class uq(torch.autograd.Function):

        def forward(ctx, input, alpha):
            input_d = input/alpha                          # weights are first divided by alpha                       
            input_c = input_d.clamp(min=-1, max=1)       # then clipped to [-1,1]
            sign = input_c.sign()
            input_abs = input_c.abs()
            input_q = uniform_quant(input_abs, b).mul(sign)
            ctx.save_for_backward(input, input_q)
            input_q_out = input_q.mul(alpha)               # rescale to the original range
            return input_q_out

    return uq().apply



class weight_quantize_fn(nn.Module):
    def __init__(self, w_bit):
        super(weight_quantize_fn, self).__init__()
        self.w_bit = w_bit-1
        self.weight_q = weight_quantization(b=self.w_bit)
        self.wgt_alpha = 0.0

    def forward(self, weight):
        weight_q = self.weight_q(weight, self.wgt_alpha)
        
        return weight_q
    

In [9]:
# 4 bits weight quantization
i = 0
output_each_layer_diff_4bit = []
mse = nn.MSELoss()

for layer in model_resnet20.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print(layer)
        #print(layer.weight.max())
        #print(layer.weight.min())
        #print(layer.weight.std())
        cur_weight = layer.weight
        # slide lecture7_Oct18-2 p.4 "𝛼 can be 1 - 3*sigma in the distribution rather than maximum value"
        w_alpha = 1 - 3 * cur_weight.std()
        #w_alpha = 4.0
        # cliping value
        w_bits = 4
        
        weight_quant = weight_quantize_fn(w_bit= w_bits)  ## define quant function
        weight_quant.wgt_alpha = torch.tensor(w_alpha)
        w_quant      = weight_quant(cur_weight)
        w_delta      = w_alpha/(2**(w_bits-1)-1)
        w_int        = w_quant/w_delta
        
        
        
        #layer.cuda()
        #conv2.cuda()
        
        x = save_output_resnet20.outputs[i][0]
        i = i + 1
         
        gnd_truth = layer(x)
        
        layer.weight = torch.nn.Parameter(w_int)
        psum_quant = layer(x)
        
        layer.weight = cur_weight

        diff = mse(psum_quant, gnd_truth)
        print(diff)
        #print(psum_quant)
        #output_each_layer_diff_4bit.append(diff.cpu().numpy)
        
        

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(541.6913, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(4.2018, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(6.9391, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(14.5794, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(4.4139, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn

/tmp/ipykernel_9866/1782699776.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weight_quant.wgt_alpha = torch.tensor(w_alpha)


In [10]:
# 8 bits weight quantization

i = 0
output_each_layer_diff_8bit = []
mse = nn.MSELoss()

for layer in model_resnet20.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print(layer)
        #print(layer.weight.max())
        #print(layer.weight.min())
        #print(layer.weight.std())
        cur_weight = layer.weight
        # slide lecture7_Oct18-2 p.4 "𝛼 can be 1 - 3*sigma in the distribution rather than maximum value"
        w_alpha = 1 - 3 * cur_weight.std()
        #w_alpha = 4.0
        # cliping value
        w_bits = 8
        
        weight_quant = weight_quantize_fn(w_bit= w_bits)  ## define quant function
        weight_quant.wgt_alpha = torch.tensor(w_alpha)
        w_quant      = weight_quant(cur_weight)
        w_delta      = w_alpha/(2**(w_bits-1)-1)
        w_int        = w_quant/w_delta
        
        
        
        #layer.cuda()
        #conv2.cuda()
        
        x = save_output_resnet20.outputs[i][0]
        i = i + 1
         
        gnd_truth = layer(x)
        
        layer.weight = torch.nn.Parameter(w_int)
        psum_quant = layer(x)
        
        layer.weight = cur_weight

        diff = mse(psum_quant, gnd_truth)
        print(diff)
        #print(psum_quant)
        #output_each_layer_diff_8bit.append(diff.cpu().numpy())

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(155112.7344, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(2.2264, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(0.1956, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(2.1060, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
tensor(0.1719, device='cuda:0', grad_fn=<MseLossBackward>)
QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_

/tmp/ipykernel_9866/1771238899.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weight_quant.wgt_alpha = torch.tensor(w_alpha)


In [ ]:
#Observation: The magnitude of mse for 8 bit is smaller than 4 bit. 8 bit quantization results in smaller discrepancy than 4 bit. 